# 2 - Redes recurrentes orientadas al procesamiento de lenguaje natural

**Sumario**

1. Introducción
2. Aprendizaje por refuerzo
3. Deep Reinforcement Learning
4. Construyendo un agente mediante una red DQNN

## 2.1 - Introducción

## 2.2 - Aprendizaje por refuerzo

### 2.2.1 - Cadena de Markov

### 2.2.2 - Proceso de decisión de Markov

### 2.2.3 - Funciones de valor

### 2.2.4 - Función de Bellman

### 2.2.5 - Algoritmos tradicionales

### 2.2.6 - Algoritmos basados en tablas

## 2.3 - Deep Reinforcement Learning

## 2.4 - Construyendo un agente mediante una red DQNN

### 2.4.1 - Importación de librerías

### 2.4.2 - Construcción del agente

### 2.4.3 - Construcción del método `main()`